# Entregable 1 



In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Mapas
import geopandas as gpd

plt.rcParams.update({
    "figure.dpi": 110,
    "savefig.dpi": 110,
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "legend.fontsize": 9,
})

sns.set_theme(style="whitegrid")  


data_dir = Path(r"D:/Descargas/datos")  

data_dir


In [ ]:
# === Carga y preparación de datos (equivalente a read_nasa_data en R) ===
def read_nasa_data(file_name: str, var_name: str, folder: Path) -> pd.DataFrame:
    # Lee un CSV de NASA POWER y devuelve:
    # YEAR, LAT, LON, JAN..DEC, ANN y una columna 'Variable'
    path = folder / file_name
    if not path.exists():
        raise FileNotFoundError(
            f"No encuentro el archivo: {path}\n"
            "Revisa que 'data_dir' apunte a la carpeta correcta y que el nombre del CSV coincida."
        )
    df = pd.read_csv(path)

    # En el Rmd se selecciona: YEAR, LAT, LON, JAN:DEC, ANN
    month_cols = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
    keep_cols = ["YEAR","LAT","LON", *month_cols, "ANN"]
    missing = [c for c in keep_cols if c not in df.columns]
    if missing:
        raise ValueError(f"El archivo {file_name} no trae estas columnas esperadas: {missing}")

    df = df[keep_cols].copy()
    df["Variable"] = var_name
    return df

ws10m  = read_nasa_data("WS10M.csv", "Velocidad del Viento (m/s)", data_dir)
prec   = read_nasa_data("PRECTOTCORR_SUM.csv", "Precipitación (mm)", data_dir)
rh2m   = read_nasa_data("RH2M.csv", "Humedad Relativa (%)", data_dir)
t2m    = read_nasa_data("T2M.csv", "Temperatura (°C)", data_dir)
allsky = read_nasa_data("ALLSKY_SFC_SW_DWN.csv", "Irradiancia Solar (kWh/m2/day)", data_dir)

# Unir todos los datos (equivalente a bind_rows)
df_all = pd.concat([ws10m, prec, rh2m, t2m, allsky], ignore_index=True)

df_all.head()


## Descripción de los Datos

En este trabajo se analizan variables meteorológicas obtenidas de la plataforma oficial NASA POWER Data Access Viewer, la cual proporciona datos climáticos históricos basados en observaciones satelitales y modelos globales.
Los datos corresponden a la región del Caribe colombiano y fueron descargados utilizando un rectángulo geográfico definido por las siguientes coordenadas:
Latitud inferior: 7° N
Latitud superior: 12.8° N
Longitud izquierda: 77° O
Longitud derecha: 71° O

Esta región cubre aproximadamente la zona costera norte de Colombia, incluyendo áreas cercanas a Barranquilla, La Guajira, Magdalena y partes del Caribe occidental.
NASA POWER limita la descarga a áreas máximas de 10° × 10°, generando cerca de 1,000 puntos de grilla, lo que permite analizar la variabilidad espacial del clima en la región.

Las variables estudiadas incluyen:
Temperatura a 2 m (T2M)
Precipitación total corregida (PRECTOTCORR_SUM)
Humedad relativa a 2 m (RH2M)
Velocidad del viento a 10 m (WS10M)
Irradiancia solar superficial (ALLSKY_SFC_SW_DWN)
Estas variables son fundamentales para el análisis climático regional, estudios de energía solar y eólica, agricultura y evaluación de cambios climáticos en el Caribe colombiano.

## Descripción de las Variables Analizadas

En este análisis exploratorio se estudian cinco parámetros clave descargados de NASA POWER para la región del Caribe colombiano:

- **Temperatura a 2 metros (T2M):** temperatura del aire a 2 m sobre superficie (°C).
- **Precipitación (PRECTOTCORR_SUM):** suma total de lluvia corregida por sesgos (mm).
- **Humedad Relativa a 2 metros (RH2M):** humedad relativa del aire (%) a 2 m.
- **Velocidad del Viento a 10 metros (WS10M):** velocidad del viento a 10 m (m/s).
- **Irradiancia Solar (ALLSKY_SFC_SW_DWN):** radiación solar de onda corta en superficie, cielo despejado + nublado (kWh/m²/día).


## Análisis de Tendencia Histórica (Interanual)

In [ ]:
# === Análisis de Tendencia Histórica (Interanual) ===
df_trend = (
    df_all.groupby(["YEAR", "Variable"], as_index=False)
          .agg(Promedio_Anual=("ANN", "mean"))
)

df_trend.head()


In [ ]:
variables = df_trend["Variable"].unique().tolist()
ncol = 2
nrow = int(np.ceil(len(variables) / ncol))

fig, axes = plt.subplots(nrow, ncol, figsize=(12, 3.8*nrow), sharex=True)
axes = np.array(axes).reshape(-1)

for i, var in enumerate(variables):
    ax = axes[i]
    d = df_trend[df_trend["Variable"] == var].sort_values("YEAR")

    ax.plot(d["YEAR"], d["Promedio_Anual"], linewidth=1)

    x = d["YEAR"].to_numpy()
    y = d["Promedio_Anual"].to_numpy()
    mask = np.isfinite(x) & np.isfinite(y)
    if mask.sum() >= 2:
        coef = np.polyfit(x[mask], y[mask], 1)
        yhat = np.polyval(coef, x)
        ax.plot(x, yhat, linestyle="--", linewidth=0.9, color="black")

    ax.set_title(var, fontweight="bold")
    ax.set_xlabel("Año")
    ax.set_ylabel("Valor Promedio")
    ax.grid(True, alpha=0.25)

for j in range(len(variables), len(axes)):
    axes[j].axis("off")

fig.suptitle("Evolución Interanual (1984 - 2025)", y=1.02, fontsize=14, fontweight="bold")
fig.text(0.5, 0.995, "Tendencia lineal marcada en línea negra punteada", ha="center", va="top", fontsize=10)
plt.tight_layout()
plt.show()


La gráfica presenta la evolución interanual de cinco variables meteorológicas clave en el Caribe colombiano entre 1984 y 2025, incluyendo temperatura, precipitación, humedad relativa, velocidad del viento e irradiancia solar.

Se observa una tendencia creciente en la temperatura, consistente con el calentamiento global observado en diversas regiones tropicales. La irradiancia solar también muestra un ligero incremento, lo que puede estar relacionado con variaciones en la nubosidad o cambios atmosféricos regionales.

La humedad relativa presenta variaciones moderadas a lo largo del tiempo, con una leve tendencia decreciente, mientras que la precipitación exhibe una alta variabilidad interanual, característica del clima del Caribe colombiano influenciado por fenómenos como El Niño y La Niña.

Por su parte, la velocidad del viento muestra fluctuaciones sin una tendencia clara, indicando estabilidad relativa en este parámetro en la región estudiada.

En conjunto, los resultados sugieren cambios graduales en temperatura e irradiancia solar, junto con variabilidad natural en precipitación y viento, coherente con el comportamiento climático esperado en el Caribe colombiano.


## Ciclo Estacional (Mes a Mes)

In [ ]:
month_cols = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]

df_seasonal_long = (
    df_all.drop(columns=["ANN"])
          .melt(id_vars=["YEAR","LAT","LON","Variable"], value_vars=month_cols,
                var_name="Mes", value_name="Valor")
)

df_seasonal = (
    df_seasonal_long.groupby(["Mes","Variable"], as_index=False)
                    .agg(Media_Mensual=("Valor","mean"))
)

df_seasonal["Mes"] = pd.Categorical(df_seasonal["Mes"], categories=month_cols, ordered=True)
df_seasonal = df_seasonal.sort_values(["Variable","Mes"])

df_seasonal.head()


In [ ]:
variables = df_seasonal["Variable"].unique().tolist()
ncol = 2
nrow = int(np.ceil(len(variables) / ncol))

cmap = mpl.cm.get_cmap("viridis", len(variables))
var_color = {v: cmap(i) for i, v in enumerate(variables)}

fig, axes = plt.subplots(nrow, ncol, figsize=(12, 3.8*nrow), sharex=True)
axes = np.array(axes).reshape(-1)

for i, var in enumerate(variables):
    ax = axes[i]
    d = df_seasonal[df_seasonal["Variable"] == var].sort_values("Mes")
    ax.bar(d["Mes"].astype(str), d["Media_Mensual"], alpha=0.85, color=var_color[var])

    ax.set_title(var, fontweight="bold")
    ax.set_xlabel("Mes del Año")
    ax.set_ylabel("Valor Promedio Mensual")
    ax.tick_params(axis="x", rotation=45)
    ax.grid(True, axis="y", alpha=0.25)

for j in range(len(variables), len(axes)):
    axes[j].axis("off")

fig.suptitle("Comportamiento Estacional Promedio", y=1.02, fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()


La gráfica presenta el promedio mensual de las principales variables meteorológicas en el Caribe colombiano, permitiendo identificar patrones estacionales característicos de la región.

La precipitación muestra un aumento notable entre mayo y noviembre, con picos hacia septiembre y octubre, lo cual coincide con la temporada de lluvias típica del Caribe colombiano. En contraste, los meses de enero a marzo presentan menores niveles de precipitación, correspondientes a la temporada seca.

La humedad relativa sigue un patrón similar al de la precipitación, con valores más altos durante los meses lluviosos y menores en la temporada seca.

La temperatura se mantiene relativamente estable a lo largo del año, reflejando el clima tropical de la región, con ligeras variaciones entre meses.

La irradiancia solar presenta valores mayores durante los meses secos, cuando la nubosidad es menor, y disminuye en la temporada de lluvias.

Por último, la velocidad del viento muestra variaciones moderadas, con valores más altos en los primeros meses del año, posiblemente asociados a los vientos alisios característicos del Caribe.


## Análisis Espacial (Mapas de la Región)

In [ ]:
df_spatial = (
    df_all.groupby(["LAT","LON","Variable"], as_index=False)
          .agg(Media_Historica=("ANN","mean"))
)

world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world = world.to_crs("EPSG:4326")

df_spatial.head()


In [ ]:
xmin, xmax = -77, -71
ymin, ymax = 7, 12.8

for var in df_spatial["Variable"].unique():
    d = df_spatial[df_spatial["Variable"] == var].copy()

    grid = d.pivot(index="LAT", columns="LON", values="Media_Historica").sort_index()
    lats = grid.index.to_numpy()
    lons = grid.columns.to_numpy()
    Z = grid.to_numpy()

    fig, ax = plt.subplots(figsize=(9, 6))

    world.plot(ax=ax, color="gray95", edgecolor="gray70", linewidth=0.4)

    im = ax.imshow(
        Z,
        extent=[lons.min(), lons.max(), lats.min(), lats.max()],
        origin="lower",
        alpha=0.55,
        cmap="viridis",
        aspect="auto",
        interpolation="nearest"
    )

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel("Longitud")
    ax.set_ylabel("Latitud")
    ax.set_title(f"NASA POWER - {var}", fontweight="bold")
    ax.text(0.5, 1.01, "Caribe Colombiano", transform=ax.transAxes, ha="center", va="bottom", fontsize=10)

    cbar = plt.colorbar(im, ax=ax, fraction=0.03, pad=0.02)
    cbar.set_label(var)

    ax.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.show()


## Distribución Anual de Variables Climáticas

In [ ]:
vir = mpl.cm.get_cmap("viridis")
box_color = vir(0.55)

for var in df_all["Variable"].unique():
    d = df_all[df_all["Variable"] == var]["ANN"].dropna()

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.boxplot(
        d.values,
        patch_artist=True,
        boxprops=dict(facecolor=box_color, alpha=0.7),
        medianprops=dict(color="black"),
        whiskerprops=dict(color="black"),
        capprops=dict(color="black"),
        flierprops=dict(marker='o', markersize=4, markerfacecolor="red", markeredgecolor="red", alpha=0.8),
    )
    ax.set_title(f"Boxplot - {var}", fontweight="bold")
    ax.set_xlabel("")
    ax.set_ylabel("Valor Anual")
    ax.set_xticks([])
    ax.grid(True, axis="y", alpha=0.25)
    plt.tight_layout()
    plt.show()


## Boxplot mensual por variable

In [ ]:
month_cols = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]

df_mes = df_all.melt(
    id_vars=["YEAR","LAT","LON","Variable","ANN"],
    value_vars=month_cols,
    var_name="Mes",
    value_name="Valor"
)
df_mes["Mes"] = pd.Categorical(df_mes["Mes"], categories=month_cols, ordered=True)

for var in df_mes["Variable"].unique():
    d = df_mes[df_mes["Variable"] == var].copy()

    fig, ax = plt.subplots(figsize=(10, 4))
    sns.boxplot(data=d, x="Mes", y="Valor", ax=ax, color="orange")
    ax.set_title(f"Distribución mensual - {var}", fontweight="bold")
    ax.set_xlabel("Mes")
    ax.set_ylabel("Valor")
    ax.tick_params(axis="x", rotation=45)
    ax.grid(True, axis="y", alpha=0.25)
    plt.tight_layout()
    plt.show()


## Matriz de Correlación de Variables

La matriz de correlación permite analizar la relación entre temperatura, precipitación, humedad relativa, velocidad del viento e irradiancia solar en la región Caribe colombiana.

Se observa una alta correlación positiva (0.88) entre la humedad relativa y la precipitación, lo cual es consistente con el comportamiento climático de la región, donde los periodos de lluvia están asociados a mayor humedad atmosférica.

También se identifican correlaciones negativas entre temperatura y humedad (-0.71), así como entre irradiancia solar y humedad (-0.52). Esto indica que en días con mayor radiación solar y temperatura, la humedad tiende a disminuir, reflejando condiciones más secas.

La irradiancia solar presenta una correlación moderada positiva con la temperatura (0.47) y con la velocidad del viento (0.53), lo que sugiere que los días más despejados suelen coincidir con mayor radiación y temperaturas más altas.

Por otro lado, la velocidad del viento muestra una correlación casi nula con la temperatura (-0.02), indicando que estos fenómenos son relativamente independientes en la región estudiada.

En general, las correlaciones observadas coinciden con el comportamiento climático esperado en el Caribe colombiano, caracterizado por periodos húmedos con alta precipitación y periodos secos con mayor radiación solar.


In [ ]:
df_cor = (
    df_trend.pivot(index="YEAR", columns="Variable", values="Promedio_Anual")
            .reset_index(drop=True)
)

matriz_cor = df_cor.corr(numeric_only=True)
matriz_cor


In [ ]:
from matplotlib.patches import Ellipse

def corrplot_ellipses(corr: pd.DataFrame, title: str = ""):
    labels = corr.columns.tolist()
    n = len(labels)

    fig, ax = plt.subplots(figsize=(1.2*n + 2, 1.0*n + 2))
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.invert_yaxis()
    ax.set_aspect("equal")

    cmap = mpl.cm.get_cmap("RdBu_r")
    norm = mpl.colors.Normalize(vmin=-1, vmax=1)

    for i in range(n):
        for j in range(n):
            if j <= i:
                continue
            r = corr.iloc[i, j]
            if pd.isna(r):
                continue

            width = 0.9
            height = 0.9 * (1 - abs(r)) + 0.05
            angle = 45 if r >= 0 else -45

            e = Ellipse(
                (j + 0.5, i + 0.5),
                width=width,
                height=height,
                angle=angle,
                facecolor=cmap(norm(r)),
                edgecolor="black",
                linewidth=0.6
            )
            ax.add_patch(e)

            ax.text(j + 0.5, i + 0.5, f"{r:.2f}", ha="center", va="center", fontsize=9, color="black")

    ax.set_xticks(np.arange(n) + 0.5)
    ax.set_yticks(np.arange(n) + 0.5)
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_yticklabels(labels)

    for k in range(n + 1):
        ax.axhline(k, color="black", linewidth=0.3, alpha=0.2)
        ax.axvline(k, color="black", linewidth=0.3, alpha=0.2)

    ax.set_title(title, pad=20, fontweight="bold")
    plt.tight_layout()
    plt.show()

corrplot_ellipses(matriz_cor, title="Relación entre Variables Meteorológicas")
